In [1]:
import pdfplumber
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pickle
import numpy as np
from scipy import spatial

In [2]:
pdf = pdfplumber.open("Modulhandbuch_FS-MADS-60_Data-Science_SGo_19.09.2019.pdf")
pages = pdf.pages[0:]

In [3]:
for page in pages[10:14]:
    print(page.extract_text())
    print("---------------------")
    print(page.extract_table())

 
 
 
Module DLMDSUCE 
Use Case and Evaluation 
5 ECTS 
 
 
 
Minimum duration of the module:   1 semester 
 
Admission requirements:     Advanced Mathematics (DLMDSAM01) 
 
11/78 
---------------------
None
Module Description 
 
Module Title:  Use Case and Evaluation 
Module No.:  Semester / Term:  Duration:  
DLMDSUCE  1st semester  Minimum 1 semester 
Module Type(s):  Compulsory  Regularly Offered in: 
WS / SS 
Workload: 150 h  Credit Points: 5 
Admission Requirements:  Language of Instruction: 
  Advanced Mathematics (DLMDSAM01)  English 
Contributing Courses to Module:  Workload: 
  Use Case and Evaluation (DLMDSUCE01)  Self-study: 110 h 
Self-examination: 20 h 
Tutorials: 20 h 
Course Coordinator(s) / Tutor(s):  Module Director: 
Please see the current list of tutors on the Learning  N.N. Professur für Artificial 
Management System.  Intelligence 
References to Other Programs:  References to Other Modules in 
the Program: 
  Master Artificial Intelligence (120 ECTS) 
  Projec

In [4]:
long_string = ""
for page in pages:
    long_string+=page.extract_text()

In [5]:
def extract_objective_df(pdf):
    df = pd.DataFrame(columns = ['objective','module','study_program'])
    pages = pdf.pages[0:]
    long_string = ""
    for page in pages:
        long_string+=page.extract_text()
    for module in long_string.split('Module Title:  '): #getting full text of each module
        course_objectives = module.split('On successful completion of this course, students will be able to: ')
        if len(course_objectives)>1: #avoid empty strings

            #Module title
            title = module.split('\n')[0]

            #Module objtectives
            objectives = course_objectives[1]
            objectives = objectives.split('Teaching Methods')[0]
            if (objectives[-4:]) =='/78 ':
                objectives = objectives[:-6]
            objectives = objectives.encode()
            objectives = objectives.decode()
            objectives = objectives.replace('\n',"")
            objectives = objectives.replace('. ',".")
            objectives = objectives.split('\uf0a7  ')[1:] #[0] is always empty because of the splitting
            for objective in objectives:
                row_list = [objective,title,"DS_60"]
                df.loc[len(df)] = row_list #because of 0 indexing we can write to new row at len(df)

    return(df)

In [6]:
df=extract_objective_df(pdf)

In [7]:
def preprocess_for_bert(string):
    string = string.strip("\n.’:")
    string = string.strip("’")
    string = string.strip("\\n")
    string = string.replace("/"," ")
    return(string)

In [9]:
k21 = pickle.load(open('../Experiments/k21.sav', 'rb'))
objectives = []
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
for o in df['objective'].to_list():
    objectives.append(preprocess_for_bert(o))
embeddings = model.encode(objectives)
clusters =  k21.predict(np.array(embeddings.tolist()))
df['clusters']=clusters

In [10]:
k20_bigrams = pickle.load(open('../Experiments/k20_bigrams', 'rb'))

In [11]:
bigrams = []
for cluster in clusters:
    bigrams.append(k20_bigrams[cluster])
df['bigrams_k20']=bigrams

In [12]:
import dtale
#handy pandas visualizer, when on local Jupyter Server just run dtale.show(df)
#from Docker container visual is available at http://localhost:40000/
d=dtale.show(df)

2021-09-20 20:10:34,104 - INFO     - NumExpr defaulting to 2 threads.


In [13]:
df

,objective,module,study_program,clusters,bigrams_k20
0,understand the fundamental building blocks of ...,Advanced Statistics,DS_60,11,"develop processes, processes tools, tools moni..."
1,analyze stochastic data in terms of the underl...,Advanced Statistics,DS_60,11,"develop processes, processes tools, tools moni..."
2,utilize Bayesian statistics techniques.,Advanced Statistics,DS_60,11,"develop processes, processes tools, tools moni..."
3,summarize the properties of observed data usin...,Advanced Statistics,DS_60,11,"develop processes, processes tools, tools moni..."
4,apply data visualization techniques to design ...,Advanced Statistics,DS_60,15,"data visualization, large datasets, experience..."
5,evaluate model parameters using parameter esti...,Advanced Statistics,DS_60,11,"develop processes, processes tools, tools moni..."
6,create hypothesis tests to discriminate betwee...,Advanced Statistics,DS_60,11,"develop processes, processes tools, tools moni..."
7,analyze use cases and their requirements regar...,Use Case and Evaluation,DS_60,17,"multiple projects, r&d projects, project team,..."
8,apply common metrics to evaluate predictions.,Use Case and Evaluation,DS_60,11,"develop processes, processes tools, tools moni..."
9,evaluate key performance indicators to asses p...,Use Case and Evaluation,DS_60,17,"multiple projects, r&d projects, project team,..."


In [14]:
missing = []
for i in range(1,21):
    if i not in clusters:
        missing.append(i)

In [15]:
print(missing)

[1, 2, 4, 5, 6, 9, 10, 12, 14, 19, 20]


In [16]:
for miss in missing:
    print("Cluster "+str(miss)+" "+k20_bigrams[miss])
    print("--------------------")

Cluster 1 experience demonstrated, scientific journals,, scientific cv, cv contact, contact details;, scientific results, present scientific, scientific technical, drug discovery, discovery early, 
--------------------
Cluster 2 team player, ability work, communication skills, team environment, excellent communication, collaboration skills, skills ability, within team, communication teamwork, good communication, 
--------------------
Cluster 4 communication skills, skills excellent, excellent communication, written communication, verbal written, skills good, good communication, communication skills,, strong communication, verbal communication, 
--------------------
Cluster 5 fluent english, english fluent, written spoken, spoken english, english fluency, fluent written, english skills, language skills, excellent written, written verbal, 
--------------------
Cluster 6 career development, willingness learn, new technologies, learn new, genuine willingness, learn drive, dedicated house, 

In [21]:
print(clusters[1])
print(k21.cluster_centers_[1])

11
[-2.95787523e-01  5.51723347e-03 -1.66146872e-01 -1.16115678e-01
  4.60300114e-03 -5.90354494e-02 -6.94117760e-02  2.28264128e-01
 -1.31826613e-02 -3.59030793e-02  3.49014582e-02 -5.68343080e-02
 -1.01392297e-01  7.79629968e-02 -2.44560252e-01 -1.65196933e-01
 -1.70873908e-02  1.54388833e-02 -5.72391331e-02 -9.95805005e-02
 -1.97598963e-01  1.12351004e-01  5.50027811e-02 -4.52788992e-02
 -5.05879365e-02 -4.38817294e-02 -8.50582906e-02 -1.14206341e-01
  8.44918547e-03 -1.23405965e-01  1.03064899e-01  2.19328885e-01
  5.23688586e-02 -2.05283817e-02 -2.09508790e-02  1.91141485e-02
 -6.89445910e-02  1.35948189e-01 -1.25687337e-01  4.69568680e-02
  2.99644532e-02 -8.45881356e-02 -1.07721233e-01  2.35928694e-01
  6.93839544e-02 -1.91067824e-02 -1.67886169e-02 -8.27196689e-02
 -1.43784312e-01  2.16940252e-01 -1.90278736e-01 -2.04458384e-01
 -7.27443935e-02 -1.45632717e-02 -3.23442551e-02 -6.88944949e-02
  3.93249455e-02 -7.37172095e-02 -1.99149365e-01 -5.57332709e-02
  1.68193279e-01 -1.05

In [23]:
for index, row in df.iterrows():
    #print(clusters[index])
    #print(embeddings[index])
    #k21.cluster_centers_[1]
    result = 1 - spatial.distance.cosine(k21.cluster_centers_[clusters[index]], embeddings[index])
    print(result)
    #print(df[skill].loc[index])

0.5863848356150768
0.44607176819981653
0.5466340440704457
0.5725945040092413
0.7112941742763597
0.461238169323247
0.47674905091766817
0.6374808497085602
0.6657309156584761
0.6373570323693074
0.5138530281250866
0.48351092985774424
0.6759577001767594
0.41553657044520587
0.46389930802130064
0.5434058490600228
0.5276040998703856
0.7717299364576778
0.5289487968586027
0.8034283634557409
0.5806291013167784
0.751062527881157
0.5225641028708781
0.6124271247980214
0.6713552353466742
0.6935681905688236
0.7351345043460232
0.6624459207108445
0.7432696190156434
0.5596324679998272
0.494505108793891
0.6419866779480624
0.39309218657897027
0.5651857342308726
0.7555183020513756
0.6083167635465586
0.627345647836522
0.6248438663957591
0.5553401946073683
0.45438526831016657
0.423032930885459
0.4881898061089869
0.5228245816053408
0.2878304520997107
0.2778793394328237
0.3163909340339133
0.5261502106220124
0.3282888349544828
